In [1]:
import torch
import tiktoken
from importlib.metadata import version
from torch.utils.data import Dataset, DataLoader

In [2]:
print("tiktoken version: ", version("tiktoken"))
print("pytorch version: ", version("torch"))
print("MPS accelerator available: ", torch.backends.mps.is_available())

tiktoken version:  0.9.0
pytorch version:  2.4.0
MPS accelerator available:  True


In [3]:
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride) -> None:
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [5]:
def create_dataloader_v1(text, batch_size=4, max_length=256, stride=128, 
                         shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [6]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [7]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [8]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [9]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=2, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]


In [11]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=3, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[1464, 1807, 3619,  402]]), tensor([[1807, 3619,  402,  271]])]


In [12]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]


In [13]:
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464],
        [ 367, 2885, 1464, 1807]]), tensor([[ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619]])]
[tensor([[2885, 1464, 1807, 3619],
        [1464, 1807, 3619,  402]]), tensor([[1464, 1807, 3619,  402],
        [1807, 3619,  402,  271]])]


In [14]:
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=2, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464],
        [2885, 1464, 1807, 3619]]), tensor([[ 367, 2885, 1464, 1807],
        [1464, 1807, 3619,  402]])]
[tensor([[ 1807,  3619,   402,   271],
        [  402,   271, 10899,  2138]]), tensor([[ 3619,   402,   271, 10899],
        [  271, 10899,  2138,   257]])]


In [15]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=3, shuffle=False)
data_iter = iter(dataloader)

first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1464,  1807,  3619,   402],
        [  402,   271, 10899,  2138],
        [ 2138,   257,  7026, 15632],
        [15632,   438,  2016,   257],
        [  257,   922,  5891,  1576],
        [ 1576,   438,   568,   340],
        [  340,   373,   645,  1049]]), tensor([[  367,  2885,  1464,  1807],
        [ 1807,  3619,   402,   271],
        [  271, 10899,  2138,   257],
        [  257,  7026, 15632,   438],
        [  438,  2016,   257,   922],
        [  922,  5891,  1576,   438],
        [  438,   568,   340,   373],
        [  373,   645,  1049,  5975]])]
[tensor([[ 1049,  5975,   284,   502],
        [  502,   284,  3285,   326],
        [  326,    11,   287,   262],
        [  262,  6001,   286,   465],
        [  465, 13476,    11,   339],
        [  339,   550,  5710,   465],
        [  465, 12036,    11,  6405],
        [ 6405,   257,  5527, 27075]]), tensor([[ 5975,   284,   502,   284],
        [  284,  3285,   326,    11],
    